# Features

In [2]:
import imageio.v3 as iio
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from numpy.lib.stride_tricks import as_strided
from skimage.data import astronaut

def update_blur(kernel_size):
    blurred_image = apply_convolution(original_image, kernel_size)
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(original_image.astype(np.uint8))
    plt.title("Original Image")
    plt.axis("off")
    plt.subplot(1, 2, 2)
    plt.imshow(blurred_image)
    plt.title(f"Blurred Image (Kernel: {kernel_size}x{kernel_size})")
    plt.axis("off")
    plt.show()

In [3]:
# Required packages are already imported
original_image = astronaut().astype(np.float32)

def convolve2d_custom(image, kernel):
    h, w = image.shape
    k, _ = kernel.shape
    pad = k // 2
    padded_image = np.pad(image, pad, mode='edge')
    strided_shape = (h, w, k, k)
    strided_steps = padded_image.strides + padded_image.strides
    image_windows = as_strided(padded_image, shape=strided_shape, strides=strided_steps)
    return np.tensordot(image_windows, kernel, axes=([2, 3], [0, 1]))

In [ ]:
def apply_convolution(image, kernel_size):
    if kernel_size % 2 == 0:
        kernel_size += 1
    kernel = np.ones((kernel_size, kernel_size), dtype=np.float32) / (kernel_size ** 2)
    if len(image.shape) == 3:
        filtered_image = np.stack([convolve2d_custom(image[:, :, c], kernel) for c in range(3)], axis=-1)
    else:
        filtered_image = convolve2d_custom(image, kernel)
    return np.clip(filtered_image, 0, 255).astype(np.uint8)

kernel_slider = widgets.IntSlider(min=3, max=21, step=2, value=3, description="Kernel Size")
widgets.interactive(update_blur, kernel_size=kernel_slider)

interactive(children=(IntSlider(value=3, description='Kernel Size', max=21, min=3, step=2), Output()), _dom_cl…